### このノート
- SMT（背景論理付きSATソルバー）を使いたい
  - Z3を使ってみる
  
- data : X_mushroom, y_mushroom


#### 起こったトラブルとその解決
- 選択したい特徴量の選択レベル（判定基準となる数値）が同じであったが、はじめに選んだ選択量を次回以降選ばないという制約を付けなかったために、2回目以降もはじめに選んだ特徴量を選んでしまっていた

In [1]:
!pip3 install z3-solver

    100% |################################| 29.6MB 60kB/s eta 0:00:011 6% |##                              | 2.0MB 689kB/s eta 0:00:41    52% |################                | 15.7MB 5.2MB/s eta 0:00:03    76% |########################        | 22.7MB 8.4MB/s eta 0:00:01    86% |###########################     | 25.5MB 5.8MB/s eta 0:00:01


In [2]:
import itertools
import pandas as pd
import numpy as np
from z3 import *

In [3]:
X_mushroom = pd.read_csv("../input/X_mushroom.csv", sep=',', index_col=0)
y_mushroom = pd.read_csv('../input/y_mushroom.csv', sep=',', index_col=0)['21']

In [4]:
num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]

In [5]:
selected_col_num2 = 2

In [6]:
y1_sum = sum(y_mushroom)
y1_sum

4208

In [7]:
def make_select_cols(X, y, sel_col_num):
    inner_product_list = [np.dot(X[each_col], y) for each_col in X.columns] #内積のリスト
    sort_inner_product_list = sorted(inner_product_list ,reverse=True)
    
    select_cols = []
    remain_num = sel_col_num
    
    while True:
        next_index = sel_col_num - remain_num
        next_colindexs = [j for j, x in enumerate(inner_product_list) if x == sort_inner_product_list[next_index]]
        
        if len(next_colindexs) >= remain_num:
            select_cols += next_colindexs[:remain_num]
            break
        select_cols += next_colindexs
        remain_num = remain_num - len(next_colindexs)
    
    select_cols = [str(n) for n in select_cols]
    else_cols = list(X.columns)
    for col in select_cols:
        else_cols.remove(str(col))
        
    return select_cols, else_cols 

In [8]:
select_cols, else_cols = make_select_cols(X_mushroom, y_mushroom, selected_col_num2)
select_cols

['1', '80']

In [9]:
ys = [BitVec("y%d" % i, 1) for i in range(num_samples)]

In [10]:
s = Solver()

In [11]:
for sel in select_cols:
    for els in else_cols:
        formu_plus = 0
        formu_minu = 0
        y_sum = 0
        for sum_num in range(num_samples):
            formu_plus += X_mushroom[sel][sum_num]*ys[sum_num]
            formu_minu += X_mushroom[els][sum_num]*ys[sum_num]
            y_sum += ys[sum_num]
        s.add(formu_plus - formu_minu >= 0)
        s.add(y_sum == y1_sum)

In [ ]:
s.check()
m = s.model()

In [ ]:
for v in ys:
    print(m[v].as_long())